Want to make a way to replace a galaxy_table with one made with a sham. 

In [1]:
import yaml
import numpy as np 

In [2]:
from pearce.mocks import cat_dict

In [23]:
from AbundanceMatching import *

In [6]:
fname = '/home/users/swmclau2/Git/pearce/bin/mcmc/nh_gg_mcmc_config.yaml'
with open(fname, 'r') as yamlfile:
    cfg = yaml.load(yamlfile)['data']

In [7]:
sim_cfg = cfg['sim']
obs_cfg = cfg['obs']
cov_cfg = cfg['cov']

In [8]:
cat = cat_dict[sim_cfg['simname']](**sim_cfg['sim_hps'])  # construct the specified catalog!

In [86]:
cat.load(sim_cfg['scale_factor'], **sim_cfg['sim_hps'])

In [10]:
r_bins = obs_cfg['rbins']
obs = obs_cfg['obs']

In [11]:
calc_observable = getattr(cat, 'calc_%s' % obs)

In [13]:
cat.populate()

In [2]:
smf = np.genfromtxt('/scratch/users/swmclau2/DR10_cBOSS_WISE_SMF_z0.45_0.60_M7.dat', skip_header=True)[:,0:2]

In [5]:
np.savetxt('/home/users/swmclau2/Git/pearce/bin/mcmc/DR10_cBOSS_SMF_z0.45.npy', smf)

In [34]:
halo_property = 'halo_mvir'

In [35]:
nd = float(sim_cfg['nd'])

In [36]:
af = AbundanceFunction(smf[:,0], smf[:,1], (10, 12.9))

In [37]:
scatter = 0.2
remainder = af.deconvolute(scatter, 20)

In [38]:
cat.halocat.halo_table.colnames

['halo_upid',
 'halo_vx',
 'halo_y',
 'halo_x',
 'halo_z',
 'halo_vy',
 'halo_vz',
 'halo_rs',
 'halo_rvir',
 'halo_mvir',
 'halo_id',
 'halo_nfw_conc',
 'halo_hostid',
 'halo_local_density_1',
 'halo_local_density_5',
 'halo_local_density_10',
 'halo_mvir_host_halo']

In [39]:
nd_halos = calc_number_densities(cat.halocat.halo_table[halo_property], 1000.0) #don't think this matters which one i choose here

In [65]:
catalog_w_nan = af.match(nd_halos, scatter)

In [75]:
n_obj_needed = int(nd*(1000.0**3))
catalog = cat.halocat.halo_table[~np.isnan(catalog_w_nan)]
sort_idxs = np.argsort(catalog)
final_catalog = catalog[sort_idxs[-n_obj_needed:]]

In [74]:
cat.model.mock.galaxy_table.colnames

['halo_upid',
 'halo_hostid',
 'conc_NFWmodel',
 'halo_y',
 'halo_x',
 'halo_z',
 'halo_vx',
 'halo_vy',
 'halo_vz',
 'halo_rvir',
 'halo_mvir',
 'halo_id',
 'halo_num_centrals',
 'halo_num_satellites',
 'gal_type',
 'vx',
 'host_centric_distance',
 'vy',
 'y',
 'x',
 'vz',
 'z']

In [77]:
final_catalog.colnames

['halo_upid',
 'halo_vx',
 'halo_y',
 'halo_x',
 'halo_z',
 'halo_vy',
 'halo_vz',
 'halo_rs',
 'halo_rvir',
 'halo_mvir',
 'halo_id',
 'halo_nfw_conc',
 'halo_hostid',
 'halo_local_density_1',
 'halo_local_density_5',
 'halo_local_density_10',
 'halo_mvir_host_halo']

In [72]:
len(cat.halocat.halo_table)

9551524

In [81]:
final_catalog['x'] = final_catalog['halo_x']
final_catalog['y'] = final_catalog['halo_y']
final_catalog['z'] = final_catalog['halo_z']

In [82]:
cat.model.mock.galaxy_table = final_catalog

In [84]:
y = calc_observable(r_bins)

In [85]:
y

array([  2.21972312e+04,   1.52625442e+04,   1.15148279e+04,
         8.04018003e+03,   4.61294897e+03,   2.24597609e+03,
         1.03323890e+03,   4.37374018e+02,   1.72043720e+02,
         6.13073978e+01,   2.26399959e+01,   1.02563802e+01,
         5.57525641e+00,   3.29144127e+00,   1.92670376e+00,
         1.10463617e+00,   6.33426580e-01,   3.61839184e-01])